In [13]:
from six.moves import cPickle as pickle
import numpy as np
import os
from scipy.misc import imread, imshow

In [4]:
def load_pickle(f):
    return  pickle.load(f, encoding='latin1')
    
def load_CIFAR_batch(filename):
    """ load single batch of cifar """
    with open(filename, 'rb') as f:
        datadict = load_pickle(f)
        X = datadict['data']
        Y = datadict['labels']
        X = X.reshape(10000, 3, 32, 32).transpose(0,2,3,1).astype("float")
        Y = np.array(Y)
    return X, Y

def load_CIFAR10(ROOT):
    """ load all of cifar """
    xs = []
    ys = []
    for b in range(1,6):
        f = os.path.join(ROOT, 'data_batch_%d' % (b, ))
        X, Y = load_CIFAR_batch(f)
        xs.append(X)
        ys.append(Y)    
    Xtr = np.concatenate(xs)
    Ytr = np.concatenate(ys)
    del X, Y
    Xte, Yte = load_CIFAR_batch(os.path.join(ROOT, 'test_batch'))
    return Xtr, Ytr, Xte, Yte

In [9]:
xTrain, yTrain, xTest, yTest = load_CIFAR10("Datasets/cifar-10-batches-py/")
print("X_train:", xTrain.shape)
print("Y_train:", yTrain.shape)
print("X_test:", xTest.shape)
print("Y_test:", yTest.shape)

(50000, 32, 32, 3)
(50000,)
(10000, 32, 32, 3)
(10000,)


In [17]:
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

# extract features

In [53]:
from skimage.feature import hog
from skimage.color import rgb2gray
from skimage import data, exposure
from skimage.feature import local_binary_pattern

def extract_features(mode, X_train, X_test):
    if mode == "RAW":
        X_train = X_train
        X_test = X_test
    elif mode == "HOG":
        X_train = np.array([hog(rgb2gray(image), orientations=8, pixels_per_cell=(16, 16),
                   cells_per_block=(1, 1)) for image in X_train])
        X_test = np.array([hog(rgb2gray(image), orientations=8, pixels_per_cell=(16, 16),
                   cells_per_block=(1, 1)) for image in X_test])
    elif mode == "LBP":
        X_train = np.array([local_binary_pattern(rgb2gray(image),P=1,R=2) for image in X_train])
        X_test = np.array([local_binary_pattern(rgb2gray(image),P=1,R=2) for image in X_test])
    elif mode == "SIFT":
        # TODO: cv2 in my computer has some bugs. I need to rebuild it from source.
#         gray= cv2.cvtColor(xTrain[0].astype(np.uint8),cv2.COLOR_BGR2GRAY)  
#         sift = cv2.SIFT(200) 
#         kp = sift.detect(gray,None)  
#         des = sift.compute(gray,kp)
        pass
    X_train = np.reshape(X_train, (X_train.shape[0], -1))
    X_test = np.reshape(X_test, (X_test.shape[0], -1))
    return X_train, X_test


# models

In [48]:
from sklearn import neighbors, svm
from sklearn.linear_model import LogisticRegression
def models(model_name, opt=None):
    if model_name == "knn":
        if opt is not None:
            model = neighbors.KNeighborsClassifier(opt["k"])
        else:
            model = neighbors.KNeighborsClassifier()
    elif model_name == "svm":
        model = svm.SVC()
    elif model_name == "lr":
        model = LogisticRegression()
    return model

# train and test

In [54]:
import time
# subsample, just for a fast try
xTrain = xTrain[:5000]
Y_train = yTrain[:5000]
xTest = xTest[:1000]
Y_test = yTest[:1000]

# features: hog
X_train, X_test = extract_features(mode="HOG", X_train=xTrain, X_test=xTest)
# model: knn
model = models("knn")
start = time.time()
model.fit(X_train, Y_train)
print("features: HOG, model: knn:", model.score(X_test, Y_test), "time:", time.time()-start)
# model: svm
model = models("svm")
start = time.time()
model.fit(X_train, Y_train)
print("features: HOG, model: svm:", model.score(X_test, Y_test), "time:", time.time()-start)
# model: lr
model = models("lr")
start = time.time()
model.fit(X_train, Y_train)
print("features: HOG, model: lr:", model.score(X_test, Y_test), "time:", time.time()-start)

# features: LBP
X_train, X_test = extract_features(mode="LBP", X_train=xTrain, X_test=xTest)
# model: knn
model = models("knn")
start = time.time()
model.fit(X_train, Y_train)
print("features: LBP, model: knn:", model.score(X_test, Y_test), "time:", time.time()-start)
# model: svm
model = models("svm")
start = time.time()
model.fit(X_train, Y_train)
print("features: LBP, model: svm:", model.score(X_test, Y_test), "time:", time.time()-start)
# model: lr
model = models("lr")
start = time.time()
model.fit(X_train, Y_train)
print("features: LBP, model: lr:", model.score(X_test, Y_test), "time:", time.time()-start)

# features: RAW
X_train, X_test = extract_features(mode="RAW", X_train=xTrain, X_test=xTest)
# model: knn
model = models("knn")
start = time.time()
model.fit(X_train, Y_train)
print("features: RAW, model: knn:", model.score(X_test, Y_test), "time:", time.time()-start)
# model: svm
model = models("svm")
start = time.time()
model.fit(X_train, Y_train)
print("features: RAW, model: svm:", model.score(X_test, Y_test), "time:", time.time()-start)
# model: lr
model = models("lr")
start = time.time()
model.fit(X_train, Y_train)
print("features: RAW, model: lr:", model.score(X_test, Y_test), "time:", time.time()-start)

/usr/local/lib/python3.5/dist-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


features: HOG, model: knn: 0.316 time: 0.30071091651916504
features: HOG, model: svm: 0.278 time: 2.9465811252593994
features: HOG, model: lr: 0.347 time: 0.14871549606323242
features: LBP, model: knn: 0.192 time: 8.000333786010742
features: LBP, model: svm: 0.215 time: 49.28135538101196
features: LBP, model: lr: 0.133 time: 12.523408889770508
features: RAW, model: knn: 0.266 time: 23.53723692893982
features: RAW, model: svm: 0.106 time: 148.2321000099182
features: RAW, model: lr: 0.207 time: 854.5061383247375


# cross validation

In [ ]:
# TODO: just try different hyparameters and try cross validation to choose better hyparameters
# TODO: use all data. Now I only use 1/10 data to train and test